In [1]:
import pandas as pd #nul
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
import json
import geopandas as gpd
from tool_function import *

In [109]:
#Sklearn
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn import svm
from sklearn.inspection import permutation_importance
from sklearn.decomposition import PCA
from sklearn.pipeline import make_pipeline
from sklearn.manifold import TSNE
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.metrics import RocCurveDisplay
from sklearn.base import clone
from sklearn import linear_model

# Data Prep

In [2]:
mutations = pd.read_pickle('../../data_mutation/geometric_mutations.pkl')

In [3]:
mutations = mutation_process(mutations)
mutations = niveau_center_connexion(mutations)

hi!


In [85]:
# We only want newly built
mutations = mutations[mutations.libnatmut.isin(["Vente en l'état futur d'achèvement", 'Vente'])].copy()

#We only want appartements
mutations_apt = mutations[(mutations.nblocmai == 0) & 
(mutations.nblocdep == 0) &
(mutations.nblocact == 0) &
(mutations.nblocapt > 0) ].copy()

#'libtypbien' to erase?
#'first_idpar' to erase?
mutations_apt = mutations[
  ['idmutation', 'idmutinvar', 'idopendata', 'idnatmut', 
      'datemut', 'coddep', 'libnatmut', 'vefa', 'valeurfonc',
       'nbdispo', 'nblot', 'nbcomm', 'l_codinsee', 
       'nbpar', 'nbparmut', 'nbsuf', 'sterr',
       'nbvolmut', 'nblocmut', 'nblocapt',
       'nbapt1pp', 'nbapt2pp', 'nbapt3pp', 'nbapt4pp',
       'nbapt5pp', 'sbatapt', 'sapt1pp', 'sapt2pp',
       'sapt3pp', 'sapt4pp', 'sapt5pp', 
       'codtypbien', 'department_code', 'libtypbien',
       'geometry', 'centroid', 'month', 'year', 'day',
       'nivcentr']
       ].copy()

#remove rows with a valeurfonc lower than 5k? 
mutations_apt = mutations_apt[mutations_apt.valeurfonc > 5000].copy()

In [ ]:
'idmutation', 
      'datemut', 'coddep', 'libnatmut', 'vefa',
       'nbdispo', 'nblot', 'nbcomm', 'l_codinsee', 
       'nbpar', 'nbparmut', 'nbsuf', 'sterr',
       'nbvolmut', 'nblocmut', 'nblocapt',
       'nbapt1pp', 'nbapt2pp', 'nbapt3pp', 'nbapt4pp',
       'nbapt5pp', 'sbatapt', 'sapt1pp', 'sapt2pp',
       'sapt3pp', 'sapt4pp', 'sapt5pp', 
       'codtypbien', 'department_code', 'libtypbien',
       'month', 'year', 'day',
       'nivcentr'

one_hot = ['libnatmut', 'libtypbien']
y = ['valeurfonc']

# Separate X and Y

In [86]:
X = mutations_apt[[
    'idmutation', 
      'datemut', 'coddep', 'libnatmut', 'vefa',
       'nbdispo', 'nblot', 'nbcomm', 'l_codinsee', 
       'nbpar', 'nbparmut', 'nbsuf', 'sterr',
       'nbvolmut', 'nblocmut', 'nblocapt',
       'nbapt1pp', 'nbapt2pp', 'nbapt3pp', 'nbapt4pp',
       'nbapt5pp', 'sbatapt', 'sapt1pp', 'sapt2pp',
       'sapt3pp', 'sapt4pp', 'sapt5pp', 
       'codtypbien', 'department_code', 'libtypbien',
       'month', 'year', 'day',
       'nivcentr'
]].copy()
Y = mutations_apt[['valeurfonc']].copy()
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=42)

In [92]:
ohe = OneHotEncoder(handle_unknown='ignore').fit(X_train)
scalery = StandardScaler().fit(y_train)
y_train = scalery.transform(y_train)
#scalery.inverse_transform(scalery.transform(y_train))

In [ ]:
def fit(pipeline, X_train, Y_train):
    

In [104]:
clf_sgd = linear_model.SGDRegressor(loss='squared_error', penalty=None, random_state=42) 

In [105]:
pipeline_sgd = make_pipeline(ohe, clf_sgd)

In [107]:
pipeline_sgd.fit(X_train, y_train.ravel())

Pipeline(steps=[('onehotencoder', OneHotEncoder(handle_unknown='ignore')),
                ('sgdregressor', SGDRegressor(penalty=None, random_state=42))])

In [108]:
pred = pipeline_sgd.predict(X_test)

In [110]:
pred

array([-0.14112803,  0.14930212,  0.21705521, ...,  0.13496249,
        0.03437364, -0.02573457])